<a href="https://colab.research.google.com/github/ppppxxzz/DC_BOT_colab/blob/main/LawBot02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 掛載 Google Drive 到指定目錄
from google.colab import drive
import os
def mount_google_drive():
    """掛載 Google Drive 以存取其檔案。"""
    print("正在掛載 Google Drive...")
    drive.mount('/content/drive/MyDrive/Colab Notebooks/DC_BOT')
    print("Google Drive 掛載成功！檔案位於 '/content/drive/My Drive'。")

# 設定根目錄（您所有檔案均放在此目錄下）
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/DC_BOT'
os.chdir(BASE_PATH)
current_directory = os.getcwd()
print("當前工作目錄:", current_directory)

# 安裝必要套件（如果尚未安裝的話）
!pip install --upgrade openai==0.28.0 faiss-cpu discord.py

# 匯入所需模組
import json
import openai
import numpy as np
import faiss

# 定義讀取 JSON 檔案的共用函數
def load_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"錯誤：找不到 {file_path}，請確認檔案存在。")
        exit(1)
    except json.JSONDecodeError:
        print(f"錯誤：無法解析 {file_path} 的 JSON，請檢查格式。")
        exit(1)

# 載入設定檔（此處的 config.json 必須放在 BASE_PATH 下）
CONFIG_FILE = os.path.join(BASE_PATH, 'config.json')
config = load_json(CONFIG_FILE)
OPENAI_API_KEY = config.get('openai_api_key')
if not OPENAI_API_KEY:
    print("錯誤：config.json 中缺少 openai_api_key 的設定。")
    exit(1)

# 設定 OpenAI API 金鑰
openai.api_key = OPENAI_API_KEY

print("環境設置完成！")


當前工作目錄: /content/drive/MyDrive/Colab Notebooks/DC_BOT
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
環境設置完成！


In [ ]:
# 設定知識庫資料夾路徑（放在 BASE_PATH 下）
KNOWLEDGE_BASE_FOLDER = os.path.join(BASE_PATH, 'knowledge_base')

# 定義取得嵌入向量的函數
def get_embedding(text):
    try:
        response = openai.Embedding.create(
            input=[text],
            model="text-embedding-ada-002"
        )
        return response['data'][0]['embedding']
    except openai.OpenAIError as e:
        print(f"OpenAI API 錯誤：{e}")
        return None

# 從知識庫資料夾中讀取所有 JSON 檔案，並整理成文件列表
def load_knowledge_base(folder_path):
    knowledge_base = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):  # 只處理 JSON 檔
            file_path = os.path.join(folder_path, file_name)
            try:
                data = load_json(file_path)
                for item in data:
                    # 假設每個條目包含 act、article 與 content 三個欄位
                    knowledge_base.append(f"{item['act']} {item['article']}: {item['content']}")
            except KeyError as e:
                print(f"{file_name} 缺少必要的鍵：{e}")
            except Exception as e:
                print(f"處理 {file_name} 時發生錯誤：{e}")
    return knowledge_base

# 載入知識庫
knowledge_base = load_knowledge_base(KNOWLEDGE_BASE_FOLDER)
if not knowledge_base:
    print("錯誤：知識庫為空，請檢查資料夾內容。")
    exit(1)

# 生成每個文件的嵌入向量
embeddings = []
total_docs = len(knowledge_base)
for idx, doc in enumerate(knowledge_base):
    embedding = get_embedding(doc)
    if embedding is not None:
        embeddings.append(embedding)
    else:
        # 若嵌入失敗，補上 1536 維零向量（text-embedding-ada-002 模型維度為 1536）
        embeddings.append([0.0] * 1536)
        print(f"[FAILURE] ({idx + 1}/{total_docs}) 嵌入失敗：{doc}")

# 轉換為 numpy 陣列
embeddings = np.array(embeddings).astype('float32')

# 儲存嵌入向量與知識庫內容到 BASE_PATH 下
np.save(os.path.join(BASE_PATH, "embeddings.npy"), embeddings)
with open(os.path.join(BASE_PATH, "knowledge_base.json"), "w", encoding="utf-8") as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=4)

print("所有嵌入向量與知識庫內容已成功儲存！")


所有嵌入向量與知識庫內容已成功儲存！


In [ ]:
import discord
from discord.ext import commands
import openai
import json
import faiss
import numpy as np
import os
import nest_asyncio
import asyncio
from datetime import datetime
import uuid

# 定義 JSON 配置檔案名稱與其他檔案路徑
CONFIG_FILE = '/content/drive/MyDrive/Colab Notebooks/DC_BOT/config.json'
EMBEDDINGS_FILE = 'embeddings.npy'  # 嵌入向量檔案名稱
CONVERSATION_LOG_FILE = 'conversation_log.json'  # 對話記錄檔案

# 讀取 JSON 檔案的共用函式
def load_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"錯誤: 找不到 {file_path}。請確保檔案存在。")
        return {}
    except json.JSONDecodeError:
        print(f"錯誤: 無法解析 {file_path} 的 JSON 格式。請檢查檔案格式。")
        return {}

def save_json(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 載入配置
config = load_json(CONFIG_FILE)
DISCORD_BOT_TOKEN = config.get('discord_bot_token')
OPENAI_API_KEY = config.get('openai_api_key')
KNOWLEDGE_BASE_FOLDER = config.get('knowledge_base_folder', '/content/drive/MyDrive/Colab Notebooks/DC_BOT/knowledge_base/')

if not DISCORD_BOT_TOKEN or not OPENAI_API_KEY:
    print("錯誤: config.json 中缺少必要的配置值。")
    exit(1)

openai.api_key = OPENAI_API_KEY

# 載入知識庫
def load_knowledge_base(folder_path):
    knowledge_base = []
    if not os.path.isdir(folder_path):
        print(f"錯誤: {folder_path} 不是一個有效的資料夾路徑。")
        exit(1)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                data = load_json(file_path)
                for item in data:
                    knowledge_base.append(f"{item['act']} {item['article']}: {item['content']}")
            except KeyError as e:
                print(f"{file_name} 缺少鍵: {e}")
            except Exception as e:
                print(f"處理 {file_name} 時發生錯誤: {e}")
    return knowledge_base

knowledge_base = load_knowledge_base(KNOWLEDGE_BASE_FOLDER)
if not knowledge_base:
    print("錯誤: 未能從知識庫中讀取到任何內容。")
    exit(1)

if not os.path.exists(EMBEDDINGS_FILE):
    print(f"錯誤: 找不到嵌入向量檔案 {EMBEDDINGS_FILE}。")
    exit(1)
embeddings = np.load(EMBEDDINGS_FILE).astype('float32')

# 建立 FAISS 索引
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("FAISS 索引建立成功。")

# 初始化 Discord 客戶端
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

def log_conversation(user_id, user_message, bot_reply, retrieved_docs):
    log_data = load_json(CONVERSATION_LOG_FILE)  # 讀取現有的對話紀錄
    conversation_id = str(uuid.uuid4())  # 生成唯一對話 ID
    timestamp = datetime.utcnow().isoformat()  # 生成 UTC 時間戳

    # 獲取當前用戶的對話次數，並產生新的增量 ID
    user_key = str(user_id)  # 確保 user_id 為字串
    if user_key not in log_data:
        log_data[user_key] = []  # 初始化該用戶的對話紀錄
        increment_id = 1  # 第一條對話，增量 ID 設為 1
    else:
        increment_id = len(log_data[user_key]) + 1  # 取該用戶的對話數 +1

    conversation_entry = {
        "conversation_id": conversation_id,  # UUID
        "increment_id": increment_id,  # 增量 ID
        "timestamp": timestamp,  # 時間戳
        "messages": [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": bot_reply}
        ],
        "retrieved_docs": retrieved_docs  # 儲存 FAISS 搜索到的前 k 筆資料
    }

    log_data[user_key].append(conversation_entry)  # 按 user_id 存入對話紀錄
    save_json(CONVERSATION_LOG_FILE, log_data)  # 儲存紀錄


@bot.event
async def on_ready():
    print(f'已登入為 {bot.user}')

@bot.command()
async def call(ctx, *, user_message: str):
    try:
        response = openai.Embedding.create(
            input=[user_message],
            model="text-embedding-ada-002"
        )
        query_embedding = response['data'][0]['embedding']
    except openai.OpenAIError as e:
        await ctx.send("抱歉，無法生成您的請求的嵌入向量。")
        print(f"OpenAI API 錯誤: {e}")
        return

    # 使用 FAISS 搜尋最相關的 3 筆資料
    D, I = index.search(np.array([query_embedding]).astype('float32'), k=3)
    print("FAISS 搜尋結果索引:", I[0])
    for i, idx in enumerate(I[0]):
        print(f"搜尋結果 {i+1}: {knowledge_base[idx]}")
        print(f"對應的嵌入向量: {embeddings[idx]}")

    retrieved_docs = [knowledge_base[i].strip() for i in I[0]]
    context_text = "\n".join(retrieved_docs)

    prompt = f"根據以下內容回答問題：\n{context_text}\n\n問題：{user_message}\n回答："

    try:
        completion = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "你是一個律師，專門提供法律諮詢。"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=600,
            temperature=0.7
        )
        reply = completion.choices[0].message.content
    except openai.OpenAIError as e:
        reply = "抱歉，我無法處理您的請求。"
        print(f"OpenAI API 錯誤: {e}")

    log_conversation(user_message, reply)
    await ctx.send(reply)

async def monitor_exit():
    while True:
        cmd = await asyncio.to_thread(input, "請輸入指令 (輸入 'exit' 可關閉程式): ")
        if cmd.strip().lower() == "exit":
            print("接收到 exit 指令，正在關閉 Bot...")
            await bot.close()
            break

nest_asyncio.apply()

if __name__ == "__main__":
    async def main():
        monitor_task = asyncio.create_task(monitor_exit())
        bot_task = asyncio.create_task(bot.start(DISCORD_BOT_TOKEN))
        done, pending = await asyncio.wait(
            [monitor_task, bot_task],
            return_when=asyncio.FIRST_COMPLETED
        )
        for task in pending:
            task.cancel()

    asyncio.run(main())


FAISS 索引建立成功。
已登入為 JiaHong#8858
FAISS 搜尋結果索引: [104 101 103]
搜尋結果 1: 刑事訴訟法 第 93-1 條: 1. 第九十一條及前條第二項所定之二十四小時，有下列情形之一者，其經過之時間不予計入。但不得有不必要之遲延：
一、因交通障礙或其他不可抗力事由所生不得已之遲滯。
二、在途解送時間。
三、依第一百條之三第一項規定不得為詢問。
四、因被告或犯罪嫌疑人身體健康突發之事由，事實上不能訊問。
五、被告或犯罪嫌疑人因表示選任辯護人之意思，而等候辯護人到場致未予訊問。但等候時間不得逾四小時。其等候第三十一條第五項律師到場致未予訊問，或因身心障礙，致無法為完全之陳述，因等候第三十五條第三項經通知陪同在場之人到場致未予訊問，亦同。
六、被告或犯罪嫌疑人須由通譯傳譯，因等候其通譯到場致未予訊問。但等候時間不得逾六小時。
七、經檢察官命具保或責付之被告，在候保或候責付中。但候保或候責付時間不得逾四小時。
八、犯罪嫌疑人經法院提審之期間。
2. 前項各款情形之經過時間內不得訊問。
3. 因第一項之法定障礙事由致二十四小時內無法移送該管法院者，檢察官聲請羈押時，並應釋明其事由。
對應的嵌入向量: [ 0.00781552 -0.00112306  0.02925811 ...  0.00559444 -0.01816942
 -0.01553752]
搜尋結果 2: 刑事訴訟法 第 91 條: 拘提或因通緝逮捕之被告，應即解送指定之處所；如二十四小時內不能達到指定之處所者，應分別其命拘提或通緝者為法院或檢察官，先行解送較近之法院或檢察機關，訊問其人有無錯誤。
對應的嵌入向量: [-0.00443701  0.00149293  0.02346397 ...  0.00827378 -0.01991709
  0.00954247]
搜尋結果 3: 刑事訴訟法 第 93 條: 1. 被告或犯罪嫌疑人因拘提或逮捕到場者，應即時訊問。
2. 偵查中經檢察官訊問後，認有羈押之必要者，應自拘提或逮捕之時起二十四小時內，以聲請書敘明犯罪事實並所犯法條及證據與羈押之理由，備具繕本並檢附卷宗及證物，聲請該管法院羈押之。但有事實足認有湮滅、偽造、變造證據或勾串共犯或證人等危害偵查目的或危害他人生命、身體之虞之卷證，應另行分卷敘明理由，請求法院以